<a href="https://colab.research.google.com/github/miyauchi-kousuke/bert/blob/main/bert_sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新しいセクション

In [ ]:
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc2/jumanpp-2.0.0-rc2.tar.xz
!tar xfv jumanpp-2.0.0-rc2.tar.xz  
%cd jumanpp-2.0.0-rc2
!mkdir bld
%cd bld
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make install -j2
!mkdir juman_bu
!mkdir juman_bu/bin
!mkdir juman_bu/libexec
!cp -rf /usr/local/bin/jumanpp ./juman_bu/bin/
!cp -rf /usr/local/libexec/jumanpp ./juman_bu/libexec/ 

In [ ]:
%cd
%cd .././content
!wget https://miyauchi-bert.s3-ap-northeast-1.amazonaws.com/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip
!unzip Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip

In [ ]:
%cd
%cd .././content
!pip install pytorch-transformers
!pip install transformers pyknp

In [ ]:
import os
import torch
from transformers import BertForMaskedLM, BertConfig, BertTokenizer
from pyknp import Juman

BASE_PATH = './Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers'
BERT_CONFIG = 'config.json'
BERT_MODEL = 'pytorch_model.bin'
VOCAVULARY_LIST = 'vocab.txt'

jumanpp = Juman()

# 形態素解析
text = 'どんなに勉強しても全然頭が良くならない'
result = jumanpp.analysis(text)
tokenized_text =[mrph.midasi for mrph in result.mrph_list()]
print(tokenized_text)

In [ ]:
# Mask 
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')

masked_index = 5 # Maskしたいtextのindex 
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

In [ ]:
%cd
%cd .././content
# Bert model
config = BertConfig.from_json_file(os.path.join(BASE_PATH, BERT_CONFIG))
model = BertForMaskedLM.from_pretrained(os.path.join(BASE_PATH, BERT_MODEL), config=config)
tokenizer = BertTokenizer(os.path.join(BASE_PATH, VOCAVULARY_LIST), do_lower_case=False, do_basic_tokenize=False)

# token化
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)